In [1]:
# https://gist.github.com/ChetnaAjmire/fa06491acc62901c68a9e010dbfd4c00


# Criando sessão spark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    LongType,
    BooleanType,
    DoubleType,
)
from pyspark.sql.functions import when, lit, isnan, length, col


In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = (
    SparkSession.builder.appName("Semana2")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()
)


In [4]:
spark


# Airport Dataset

In [5]:
schema = StructType(
    [
        StructField("faa", StringType(), True),
        StructField("name", StringType(), True),
        StructField("lat", DoubleType(), True),
        StructField("lon", DoubleType(), True),
        StructField("alt", IntegerType(), True),
        StructField("tz", DoubleType(), True),
        StructField("dst", StringType(), True),
    ]
)


In [6]:
df = spark.read.csv("../Datasets/airports.csv", header=True, schema=schema)
# ,inferSchema=True
df.show(5, False)


+---+-----------------------------+----------+-----------+----+----+---+
|faa|name                         |lat       |lon        |alt |tz  |dst|
+---+-----------------------------+----------+-----------+----+----+---+
|04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5.0|A  |
|06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5.0|A  |
|06C|Schaumburg Regional          |41.9893408|-88.1012428|801 |-6.0|A  |
|06N|Randall Airport              |41.431912 |-74.3915611|523 |-5.0|A  |
|09J|Jekyll Island Airport        |31.0744722|-81.4277778|11  |-4.0|A  |
+---+-----------------------------+----------+-----------+----+----+---+
only showing top 5 rows



In [7]:
df.printSchema()


root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: double (nullable = true)
 |-- dst: string (nullable = true)



## Pergunta 1

In [8]:
# Primeiro when checa se é nulo ou vazio
# Segundo when checa se o tamanho não está entre 3 e 5 e se não é digito alfanumérico
df = df.withColumn(
    "qa_faa",
    when((col("faa").isNull() | (col("faa") == "")), "M").when(
        (length(col("faa")) < 3)
        | (length(col("faa")) > 5)
        | ~(col("faa").rlike("^[a-zA-Z0-9]*$")),
        "F",
    ),
)


In [9]:
df.select(df.faa, df.qa_faa).filter(df.qa_faa.isNotNull()).show()


+---+------+
|faa|qa_faa|
+---+------+
+---+------+



## Pergunta 2

In [10]:
df = df.withColumn(
    "qa_name", when((col("name").isNull() | (col("name") == "")), "M")
)


In [11]:
df.select(df.name, df.qa_name).filter(df.qa_name.isNotNull()).show()


+----+-------+
|name|qa_name|
+----+-------+
+----+-------+



## Pergunta 3

In [12]:
df = df.withColumn(
    "qa_lat",
    when((col("lat").isNull() | (col("lat") == "")), "M")
    .when(
        (col("lat") < -180) | (col("lat") > 180),
        "I",
    )
    .when((col("lat").rlike("^[a-zA-Z0-9]*$")), "A"),
)
# checando
# df.select(df.lat, df.qa_lat).filter(df.lat < -180).show()
# df.select(df.lat, df.qa_lat).filter(df.lat > 180).show()


In [13]:
df.select(df.lat, df.qa_lat).filter(df.qa_lat.isNotNull()).show()


+---+------+
|lat|qa_lat|
+---+------+
+---+------+



## Pergunta 4

In [14]:
df = df.withColumn(
    "qa_lon",
    when((col("lon").isNull() | (col("lon") == "")), "M")
    .when(
        (col("lon") < -180) | (col("lon") > 180),
        "I",
    )
    .when((col("lon").rlike("^[a-zA-Z0-9]*$")), "A"),
)
# checando

df.select(df.lon, df.qa_lon).filter(df.qa_lon.isNotNull()).show()


+---+------+
|lon|qa_lon|
+---+------+
+---+------+



## Pergunta 5

In [15]:
df = df.withColumn(
    "qa_alt",
    when((col("alt").isNull() | (col("alt") == "")), "M")
    .when(
        (col("alt") < 0),
        "I",
    )
    .when((col("alt").rlike("^[a-zA-Z0-9]*$")), "A"),
)
# checando

df.select(df.alt, df.qa_alt).filter(df.qa_alt.isNotNull()).show()


+----+------+
| alt|qa_alt|
+----+------+
|1044|     A|
| 264|     A|
| 801|     A|
| 523|     A|
|  11|     A|
|1593|     A|
| 730|     A|
| 492|     A|
|1000|     A|
| 108|     A|
| 409|     A|
| 875|     A|
|1003|     A|
| 951|     A|
|1789|     A|
| 122|     A|
| 152|     A|
| 670|     A|
|1134|     A|
| 885|     A|
+----+------+
only showing top 20 rows



## Pergunta 6

In [16]:
df = df.withColumn(
    "qa_tz",
    when((col("tz").isNull() | (col("tz") == "")), "M")
    .when(
        (col("tz") < -11) | (col("tz") > 14),
        "I",
    )
    .when((col("tz").rlike("^[a-zA-Z0-9]*$")), "A"),
)
# checando

df.select(df.tz, df.qa_tz).filter(df.qa_tz.isNotNull()).show()


+---+-----+
| tz|qa_tz|
+---+-----+
+---+-----+



## Pergunta 7

In [17]:
df = df.withColumn(
    "qa_dst",
    when((col("dst").isNull() | (col("dst") == "")), "M")
    .when(~col("dst").rlike("[EASOZNU]{1}"), "C")
    .when(col("dst").rlike("[0-9]"), "N"),
)
# checando

df.select(df.dst, df.qa_dst).filter(df.qa_dst.isNotNull()).show()


+---+------+
|dst|qa_dst|
+---+------+
+---+------+



Salvando

In [18]:
"""
df.repartition(1).write.format("parquet").mode("overwrite").option(
    "header", "true"
).save("../Data/airports_qa.parquet")
"""
df = df.toPandas()
df.to_parquet("../Data/airports_qa.parquet")

# Planes Dataset

In [19]:
schema = StructType(
    [
        StructField("tailnum", StringType(), True),
        StructField("year", IntegerType(), True),
        StructField("type", StringType(), True),
        StructField("manufacturer", StringType(), True),
        StructField("model", StringType(), True),
        StructField("engines", IntegerType(), True),
        StructField("seats", IntegerType(), True),
        StructField("speed", IntegerType(), True),
        StructField("engine", StringType(), True),
    ]
)


In [20]:
df = spark.read.csv("../Datasets/planes.csv", header=True, schema=schema)
# ,inferSchema=True
df.show(5, False)


+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|type                   |manufacturer    |model   |engines|seats|speed|engine   |
+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|N102UW |1998|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |null |Turbo-fan|
|N103US |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |null |Turbo-fan|
|N104UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |null |Turbo-fan|
|N105UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |null |Turbo-fan|
|N107US |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |null |Turbo-fan|
+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
only showing top 5 rows



In [21]:
df.printSchema()


root
 |-- tailnum: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: integer (nullable = true)
 |-- engine: string (nullable = true)



Pergunta 1

In [22]:
df = df.withColumn(
    "qa_tailnum",
    when((df.tailnum).isNull() | (df.tailnum == ""), "M")
    .when((length(df.tailnum) != 5), "S")
    .when(~df.tailnum.startswith("N"), "FN")
    .when(df.tailnum.rlike("I|O|N0"), "FE")
    .when(
        ~df.tailnum.rlike("^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNPQRSTUVXWYZ]{1,2})"),
        "F",
    ),
)

df.filter(df.qa_tailnum.isin(["M", "S", "FN", "FE", "F"])).groupBy(
    "qa_tailnum"
).count().show()

df.select("tailnum", "qa_tailnum").filter(
    df.qa_tailnum.isin(["M", "S", "FN", "FE", "F"])
).show(5, False)

df.select("tailnum", "qa_tailnum").filter(df.qa_tailnum.isin(["S"])).show(
    5, False
)


+----------+-----+
|qa_tailnum|count|
+----------+-----+
|         F|   17|
|         S| 2609|
+----------+-----+

+-------+----------+
|tailnum|qa_tailnum|
+-------+----------+
|N102UW |S         |
|N103US |S         |
|N104UW |S         |
|N105UW |S         |
|N107US |S         |
+-------+----------+
only showing top 5 rows

+-------+----------+
|tailnum|qa_tailnum|
+-------+----------+
|N102UW |S         |
|N103US |S         |
|N104UW |S         |
|N105UW |S         |
|N107US |S         |
+-------+----------+
only showing top 5 rows



Pergunta 2

In [23]:
df = df.withColumn(
    "qa_year",
    when((df.year).isNull() | (df.year == ""), "M").when(df.year < 1950, "I"),
)
df.select("qa_year").groupBy("qa_year").count().show()
# df.filter(df.qa_year == 'M').show()


+-------+-----+
|qa_year|count|
+-------+-----+
|   null| 2567|
|      M|   60|
|      I|    1|
+-------+-----+



Pergunta 3

In [24]:
types = ["Fixed wing multi engine", "Fixed wing single engine", "Rotorcraft"]
df = df.withColumn(
    "qa_type",
    when((df.type).isNull() | (df.type == ""), "M").when(
        ~df.type.isin(types), "C"
    ),
)
df.select("qa_type").groupBy("qa_type").count().show()


+-------+-----+
|qa_type|count|
+-------+-----+
|   null| 2628|
+-------+-----+



Pergunta 4

In [25]:
manufacturer = [
    "AIRBUS",
    "BOEING",
    "BOMBARDIER",
    "CESSNA",
    "EMBRAER",
    "SIKORSKY",
    "CANADAIR",
    "PIPER",
    "MCDONNELL DOUGLAS",
    "CIRRUS",
    "BELL",
    "KILDALL GARY",
    "LAMBERT RICHARD",
    "BARKER JACK",
    "ROBINSON HELICOPTER",
    "GULFSTREAM",
    "MARZ BARRY",
]

df = df.withColumn(
    "qa_manufacturer",
    when((df.type).isNull() | (df.type == ""), "M").when(
        ~df.manufacturer.isin(manufacturer), "C"
    ),
)
df.select("qa_manufacturer").groupBy("qa_manufacturer").count().show()


+---------------+-----+
|qa_manufacturer|count|
+---------------+-----+
|           null| 2007|
|              C|  621|
+---------------+-----+



Pergunta 5

In [26]:
df = df.withColumn(
    "qa_model",
    when(
        ((df.manufacturer == "AIRBUS") & ~(df.model.startswith("A")))
        | ((df.manufacturer == "BOEING") & ~(df.model.startswith("7")))
        | (
            (
                (df.manufacturer == "BOMBARDIER")
                | (df.manufacturer == "CANADAIR")
            )
            & ~(df.model.startswith("CL"))
        )
        | (
            (df.manufacturer == "MCDONNELL DOUGLAS")
            & ~((df.model.startswith("MD")) | (df.model.startswith("DC")))
        ),
        "F",
    ).when((df.manufacturer).isNull() | (df.manufacturer == ""), "M"),
)
df.select("qa_model").groupBy("qa_model").count().show()


+--------+-----+
|qa_model|count|
+--------+-----+
|       F|   15|
|    null| 2613|
+--------+-----+



Pergunta 6

In [27]:
df = df.withColumn(
    "qa_engines",
    when((df.engines).isNull() | (df.engines == ""), "M")
    .when(~(df.engines.between(1, 4)), "I")
    .when(df.engines.rlike("^[a-zA-Z0-9]*$"), "A"),
)

df.select("qa_engines").groupBy("qa_engines").count().show()


+----------+-----+
|qa_engines|count|
+----------+-----+
|         A| 2628|
+----------+-----+



Pergunta 7

In [28]:
df = df.withColumn(
    "qa_seats",
    when((df.seats).isNull() | (df.seats == ""), "M")
    .when(~(df.seats.between(2, 500)), "I")
    .when(df.seats.rlike("^[a-zA-Z0-9]*$"), "A"),
)

df.select("qa_seats").groupBy("qa_seats").count().show()


+--------+-----+
|qa_seats|count|
+--------+-----+
|       A| 2628|
+--------+-----+



Pergunta 8

In [29]:
df = df.withColumn(
    "qa_speed",
    when((df.speed).isNull() | (df.speed == ""), "M")
    .when(~(df.speed.between(50, 150)), "I")
    .when(df.speed.rlike("^[a-zA-Z0-9]*$"), "A"),
)

df.select("qa_speed").groupBy("qa_speed").count().show()


+--------+-----+
|qa_speed|count|
+--------+-----+
|       M| 2622|
|       A|    6|
+--------+-----+



Pergunta 9

In [30]:
category = ["Turbo-fan", "Turbo-jet", "Turbo-prop", "Turbo-shaft", "4 Cycle"]

df = df.withColumn(
    "qa_engine",
    when((df.engine).isNull() | (df.engine == ""), "M").when(
        ~(df.engine.isin(category)), "C"
    ),
)


df.select("qa_engine").groupBy("qa_engine").count().show()


+---------+-----+
|qa_engine|count|
+---------+-----+
|     null| 2618|
|        C|   10|
+---------+-----+



Salvando

In [31]:
"""
df.repartition(1).write.format("parquet").mode("overwrite").option(
    "header", "true"
).save("../Data/planes_qa.parquet")
"""
df = df.toPandas()
df.to_parquet("../Data/planes_qa.parquet")

# Flights Dataset

In [32]:
schema = (
    StructType()
    .add("year", IntegerType(), True)
    .add("month", IntegerType(), True)
    .add("day", IntegerType(), True)
    .add("dep_time", StringType(), True)
    .add("dep_delay", IntegerType(), True)
    .add("arr_time", StringType(), True)
    .add("arr_delay", IntegerType(), True)
    .add("carrier", StringType(), True)
    .add("tailnum", StringType(), True)
    .add("flight", StringType(), True)
    .add("origin", StringType(), True)
    .add("dest", StringType(), True)
    .add("air_time", IntegerType(), True)
    .add("distance", IntegerType(), True)
    .add("hour", IntegerType(), True)
    .add("minute", IntegerType(), True)
)


In [33]:
df = spark.read.csv("../Datasets/flights.csv", header=True, schema=schema)
# ,inferSchema=True
df.show(5, False)


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |
|2014|3    |9  |1443    |-2       |1652    |2        |VX     |N847VA |755   |SEA   |SFO |111     |679     |14  |43    |
|2014|4    |9  |1705    |45       |1839    |34       |WN     |N360SW |344   |PDX   |SJC |83      |569     |17  |5     |
|2014|3    |9  |754     |-1       |1015    |1        |AS     |N612AS |522   |SEA   |BUR |127     |937     |7   |54    |
+----+-----+---+--------+---------+-----

Pergunta 1

In [34]:
df = df.withColumn(
    "qa_year_month_day",
    when(df.year.isNull(), "MY")
    .when(df.month.isNull(), "MM")
    .when(df.day.isNull(), "MD")
    .when(df.year < 1950, "IY")
    .when(~(df.month.between(1, 12)), "IM")
    .when(~(df.day.between(1, 31)), "ID")
    .when((df.month == 2) & ((col("day") > 29)), "ID"),
)

df.select("qa_year_month_day").groupBy("qa_year_month_day").count().show()


+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
|             null|10000|
+-----------------+-----+



Pergunta 2

In [35]:
df = df.withColumn(
    "qa_hour_minute",
    when(df.hour.isNull(), "MH")
    .when(df.minute.isNull(), "MM")
    .when(~(df.hour.between(0, 24)), "IH")
    .when(~(df.minute.between(0, 59)), "IM"),
)

df.select("qa_hour_minute").groupBy("qa_hour_minute").count().show()


+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
|          null| 9952|
|            MH|   48|
+--------------+-----+



Pergunta 3


In [36]:
df = df.withColumn(
    "qa_dep_arr_time",
    when((df.dep_time.isNull()) | (df.dep_time == "NA"), "MD")
    .when((df.arr_time.isNull()) | (df.arr_time == "NA"), "MA")
    .when(~df.dep_time.rlike("^([0-9]|1[0-9]|2[0-3])[0-5][0-9]$"), "FD")
    .when(~df.arr_time.rlike("^([0-9]|1[0-9]|2[0-3])[0-5][0-9]$"), "FA"),
)
df.select("qa_dep_arr_time").groupBy("qa_dep_arr_time").count().show()


+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
|           null| 9704|
|             MD|   48|
|             FA|  151|
|             MA|    7|
|             FD|   90|
+---------------+-----+



In [37]:
df.select("arr_time").groupBy("arr_time").count().orderBy(
    "count", ascending=False
).show(3)


+--------+-----+
|arr_time|count|
+--------+-----+
|      NA|   55|
|    2104|   23|
|    2101|   22|
+--------+-----+
only showing top 3 rows



Pergunta 4

In [38]:
df = df.withColumn(
    "qa_dep_arr_delay",
    when(df.dep_delay.isNull(), "MD").when(df.arr_delay.isNull(), "MA"),
)

df.select("qa_dep_arr_delay").groupBy("qa_dep_arr_delay").count().show()


+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
|            null| 9925|
|              MD|   48|
|              MA|   27|
+----------------+-----+



Pergunta 5

In [39]:
df = df.withColumn(
    "qa_carrier",
    when(df.carrier.isNull(), "M").when(
        ~(df.carrier.rlike("([0-9a-zA-Z]{2})")), "F"
    ),
)


df.select("qa_carrier").groupBy("qa_carrier").count().show()


+----------+-----+
|qa_carrier|count|
+----------+-----+
|      null|10000|
+----------+-----+



Pergunta 6

In [40]:
df = df.withColumn(
    "qa_tailnum",
    when((df.tailnum.isNull()) | (df.tailnum == "NA"), "M")
    .when((length(df.tailnum) != 5) & (length(df.tailnum) != 6), "S")
    .when((df.tailnum.rlike("^N") == False), "FN")
    .when((df.tailnum.rlike("^[IO0]") == True), "FE")
    .when((df.tailnum.rlike("^N([0-9]{1,4})([A-Z]{1,2}$)") == False), "F"),
)
df.select("qa_tailnum").groupBy("qa_tailnum").count().show()


+----------+-----+
|qa_tailnum|count|
+----------+-----+
|         F|  987|
|      null| 8997|
|         M|   14|
|        FN|    2|
+----------+-----+



Pergunta 7

In [41]:
df = df.withColumn(
    "qa_flight",
    when(df.flight.isNull(), "M").when(~df.flight.rlike("([0-9]{4})"), "F"),
)
# df.select("qa_flight","flight").filter(df.qa_flight=="F").show()
df.select("qa_flight").groupBy("qa_flight").count().show()


+---------+-----+
|qa_flight|count|
+---------+-----+
|        F| 6158|
|     null| 3842|
+---------+-----+



Pergunta 8

In [42]:
df = df.withColumn(
    "qa_origin_dest",
    when(df.origin.isNull(), "MO")
    .when(df.dest.isNull(), "MD")
    .when(~df.origin.rlike("[0-9a-zA-Z]{3}"), "FO")
    .when(~df.dest.rlike("[0-9a-zA-Z]{3}"), "FD"),
)
df.select("qa_origin_dest").groupBy("qa_origin_dest").count().show()


+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
|          null|10000|
+--------------+-----+



Pergunta 9

In [43]:
df = df.withColumn(
    "qa_air_time",
    when(df.air_time.isNull(), "M").when(~df.air_time.between(20, 500), "I"),
)

# df.select("qa_air_time","air_time").filter(df.qa_air_time=="M").show()
df.select("qa_air_time").groupBy("qa_air_time").count().show()


+-----------+-----+
|qa_air_time|count|
+-----------+-----+
|       null| 9925|
|          M|   75|
+-----------+-----+



Pergunta 10

In [44]:


df = df.withColumn("qa_distance",
                         when(df.distance.isNull(), "M")
                        .when(~df.distance.between(50, 3000), "I"))
df.select("qa_distance").groupBy("qa_distance").count().show()            

+-----------+-----+
|qa_distance|count|
+-----------+-----+
|       null|10000|
+-----------+-----+



Pergunta 11

In [45]:


df = df.withColumn("qa_distance_airtime",
                        when((df.distance.isNull())|(df.air_time.isNull()), "M")
                       .when(df.air_time >= (df.distance*0.1 + 30), "TL")
                       .when(df.air_time <= (df.distance*0.1 +10), "TS")
                       .otherwise("TR"))
df.select("qa_distance_airtime").groupBy("qa_distance_airtime").count().show()  


+-------------------+-----+
|qa_distance_airtime|count|
+-------------------+-----+
|                  M|   75|
|                 TR| 4831|
|                 TS|   67|
|                 TL| 5027|
+-------------------+-----+



Salvando

In [46]:
"""
df.repartition(1).write.format("parquet").mode("overwrite").option(
    "header", "true"
).save("../Data/flights_qa.parquet")
"""
df = df.toPandas()
df.to_parquet("../Data/flights_qa.parquet")
